In [1]:
import csv
import bcrypt
import re
from getpass import getpass
import random
import requests

In [2]:
CSV_FILE = 'regno.csv'
MAX_ATTEMPTS = 5

In [3]:
# Function to hash password
def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

# Function to check password against stored hash
def check_password(stored_hash, password):
    return bcrypt.checkpw(password.encode(), stored_hash.encode())

In [4]:
# Function to validate password criteria
def validate_password(password):
    if len(password) < 8:
        return False, "Password must be at least 8 characters long."
    if not re.search(r'[A-Z]', password):
        return False, "Password must contain at least one uppercase letter."
    if not re.search(r'[a-z]', password):
        return False, "Password must contain at least one lowercase letter."
    if not re.search(r'[0-9]', password):
        return False, "Password must contain at least one number."
    if not re.search(r'[!@#$%^&*(),.?":{}|<>]', password):
        return False, "Password must contain at least one special character."
    return True, "Password is valid."

In [5]:
# Function to read CSV data
def read_csv():
    users = []
    try:
        with open(CSV_FILE, 'r') as file:
            reader = csv.DictReader(file)
            users = list(reader)
    except FileNotFoundError:
        print("User data file not found. Creating a new file.")
    return users

In [6]:
# Function to write CSV data
def write_csv(users):
    with open(CSV_FILE, 'w', newline='') as file:
        fieldnames = ['email', 'password', 'security_question', 'security_answer']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(users)

In [7]:
# Function to prompt the user until a valid password is entered
def prompt_valid_password():
    while True:
        password = getpass("Enter a password: ").strip()
        valid, msg = validate_password(password)
        if valid:
            return password
        print(msg)  # Keep prompting if the password is not valid

In [8]:
def validate_email(email):
    # Regular expression for validating an email
    email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    
    # Check if the email matches the pattern
    if re.match(email_regex, email):
        return True, "Email is valid."
    else:
        return False, "Invalid email format. Please enter a valid email."

In [9]:
# Function to register a user
def register_user():
    while True:
        email = input("Enter your email: ").strip()
        
        # Validate email and get the validation result and message
        is_valid, message = validate_email(email)
        
        if is_valid:
            break
        else:
            print(message)
            
    # Keep prompting for a valid password
    password = prompt_valid_password()
    hashed_password = hash_password(password).decode()

    security_question = input("Enter your security question: ").strip()
    security_answer = input("Answer your security question: ").strip()
    
    # Append new user to CSV
    users = read_csv()
    users.append({'email': email, 'password': hashed_password, 'security_question': security_question, 'security_answer': security_answer})
    write_csv(users)
    print("Registration successful!")

In [10]:
# Function to reset password
def reset_password():
    email = input("Enter your registered email: ").strip()
    users = read_csv()
    user = next((u for u in users if u['email'] == email), None)
    
    if user:
        print(f"Security Question: {user['security_question']}")
        answer = input("Answer: ").strip()
        
        if answer == user['security_answer']:
            # Keep prompting for a valid password
            new_password = prompt_valid_password()
            hashed_password = hash_password(new_password).decode()

            user['password'] = hashed_password
            write_csv(users)
            print("Password reset successfully.")
        else:
            print("Incorrect answer to security question.")
    else:
        print("Email not found.")

In [11]:
def captcha():
    num1 = random.randint(1, 10)
    num2 = random.randint(1, 10)
    correct_answer = num1 + num2
    
    print(f"CAPTCHA: What is {num1} + {num2}= ?")
    user_answer = input("Enter the CAPTCHA result: ").strip()
    
    if user_answer.isdigit() and int(user_answer) == correct_answer:
        return True
    else:
        
        return False

In [12]:
# Function to login
def login():
    """ -- LogIn -- """
    attempts = 0
    users = read_csv()

    while attempts < MAX_ATTEMPTS:
        email = input("Enter your email: ").strip()
        password = getpass("Enter your password: ").strip()

        user = next((u for u in users if u['email'] == email), None)

        if user and check_password(user['password'], password):
            
            if captcha():
                print("CAPTCHA passed. Login successful!")
                return True
            else:
                print("CAPTCHA failed. Please try again.")
            
        else:
            attempts += 1
            remaining_attempts = MAX_ATTEMPTS - attempts
            print(f"Invalid email or password. {remaining_attempts} attempts remaining.")

    print("Maximum login attempts exceeded. Application locked.")
    return False

In [13]:
import requests

def search_game():
    """Search for the latest deals on a game using CheapShark API."""
    game_title = input("Enter the name of the game: ")
    
    try:
        response = requests.get(f"https://www.cheapshark.com/api/1.0/deals?title={game_title}&exact=0", timeout=10)
        response.raise_for_status()  # Raises an exception for 4xx/5xx responses
        deals = response.json()
        
        if not deals:
            print("No deals found for this game.")
            return
        
        print(f"\n-- Deals for {game_title} --")
        for deal in deals:
            print(f"Title: {deal['title']}")
            print(f"Store: {deal['storeID']}")
            print(f"Normal Price: ${deal['normalPrice']}")
            print(f"Sale Price: ${deal['salePrice']}")
            print(f"Savings: {deal['savings'][:4]}%")
            print(f"Steam App ID: {deal['steamAppID']}")
            print(f"Link: https://www.cheapshark.com/redirect?dealID={deal['dealID']}")
            print("-" * 40)
    
    except requests.exceptions.Timeout:
        print("The request timed out. Please try again later.")
    
    except requests.exceptions.ConnectionError:
        print("Network error. Please check your internet connection.")
    
    except Exception as e:
        print(f"An error occurred: {e}")

In [14]:
def post_login_menu():
    while True:
        print("\n--- Login Menu ---")
        print("1. Search Game ")
        print("2. Logout")
        
        choice = input("Choose an option: ").strip()

        if choice == '1':
            search_game()  # Display game API details
        elif choice == '2':
            print("Logging out...")
            break
        else:
            print("Invalid option. Please choose 1 or 2.")

In [ ]:
# Main application
def main():
    while True:
        print("\n -- FindMyGame Application -- ")
        print()
        print("1. Login")
        print("2. Register")
        print("3. Forgot Password")
        print("4. Exit")
        choice = input("Choose an option: ").strip()

        if choice == '1':
            if login():
                post_login_menu() 
        elif choice == '2':
            register_user()
        elif choice == '3':
            reset_password()
        elif choice == '4':
            print("Exiting........BYE👋")
            break
        else:
            print("Invalid option selected.")
        
        
main()


 -- FindMyGame Application -- 

1. Login
2. Register
3. Forgot Password
4. Exit
Choose an option: 2
Enter your email: priotosh@gmail.com
Enter a password: ········
Enter your security question: pet name
Answer your security question: sonu
Registration successful!

 -- FindMyGame Application -- 

1. Login
2. Register
3. Forgot Password
4. Exit
Choose an option: 1
Enter your email: priotosh@gmail.com
Enter your password: ········
Invalid email or password. 4 attempts remaining.
Enter your email: priotosh@gmail.com
Enter your password: ········
Invalid email or password. 3 attempts remaining.
Enter your email: Monotosh12345@
Enter your password: ········
Invalid email or password. 2 attempts remaining.
Enter your email: s
Enter your password: ········
Invalid email or password. 1 attempts remaining.
Enter your email: s
Enter your password: ········
Invalid email or password. 0 attempts remaining.
Maximum login attempts exceeded. Application locked.

 -- FindMyGame Application -- 

1. Log